In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
train_dir = os.path.join('gender/gender/', 'train')
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150, 150),class_mode='categorical')

Found 3491 images belonging to 2 classes.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
#train_datagen = ImageDataGenerator(rescale=1./255)
#train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150, 150),class_mode='categorical')
validation_dir = os.path.join('gender/gender/', 'valid')
validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(validation_dir,target_size=(150, 150),class_mode='categorical')

Found 200 images belonging to 2 classes.


In [9]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2, activation='softmax'))

In [10]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [11]:
history = model.fit_generator(train_generator,epochs=10,validation_data=validation_generator)

Epoch 1/10


C:\Users\hp\AppData\Local\Temp\ipykernel_12688\638197317.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,epochs=10,validation_data=validation_generator)


110/110 [==============================] - 82s 731ms/step - loss: 2.4075 - acc: 0.5047 - val_loss: 0.6923 - val_acc: 0.5500
Epoch 2/10
110/110 [==============================] - 82s 745ms/step - loss: 0.6943 - acc: 0.5660 - val_loss: 0.6690 - val_acc: 0.6400
Epoch 3/10
110/110 [==============================] - 94s 856ms/step - loss: 0.5919 - acc: 0.7127 - val_loss: 0.4192 - val_acc: 0.8150
Epoch 4/10
110/110 [==============================] - 107s 976ms/step - loss: 0.4327 - acc: 0.8115 - val_loss: 0.3295 - val_acc: 0.8500
Epoch 5/10
110/110 [==============================] - 91s 824ms/step - loss: 0.3878 - acc: 0.8399 - val_loss: 0.2648 - val_acc: 0.8900
Epoch 6/10
110/110 [==============================] - 120s 1s/step - loss: 0.3041 - acc: 0.8725 - val_loss: 0.3133 - val_acc: 0.9000
Epoch 7/10
110/110 [==============================] - 123s 1s/step - loss: 0.2773 - acc: 0.8889 - val_loss: 0.2708 - val_acc: 0.9050
Epoch 8/10
110/110 [==============================] - 112s 1s/step - 

In [12]:
import joblib
joblib.dump(model, 'gender_detection_model.joblib')

['gender_detection_model.joblib']

In [14]:
import cv2
import numpy as np
import joblib  # Import joblib for loading the model
from tensorflow.keras.models import load_model

# Load the Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load your trained CNN model for mask detection
# Replace 'mask_detection_model.joblib' with the path to your saved model file
model = joblib.load('gender_detection_model.joblib')

# Define the labels for mask and no-mask
labels = {0: 'Female', 1: 'Male'}

# Open a video capture stream or use an image
cap = cv2.VideoCapture(0)  # 0 for webcam, or provide a video file path

while True:
    # Read a frame from the video stream
    ret, frame = cap.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop through the detected faces
    for (x, y, w, h) in faces:
        # Extract the face ROI
        face_roi = gray[y:y + h, x:x + w]

        # Resize the face ROI to (150, 150) to match the model's input shape
        face_roi = cv2.resize(face_roi, (150, 150))

        # Convert the grayscale image to RGB by duplicating channels
        face_roi_rgb = cv2.cvtColor(face_roi, cv2.COLOR_GRAY2RGB)

        # Normalize the pixel values
        face_roi_rgb = face_roi_rgb / 255.0

        # Expand the dimensions to create a batch of size 1
        face_roi_rgb = np.expand_dims(face_roi_rgb, axis=0)

        # Make predictions using the CNN model
        prediction = model.predict(face_roi_rgb)
        label_index = np.argmax(prediction)
        label = labels[label_index]
        confidence = prediction[0][label_index]

        # Draw a rectangle around the face and display the label and confidence
        color = (0, 255, 0) if label_index == 0 else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f'{label}: {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Display the frame
    cv2.imshow('Gender Detection', frame)

    # Break the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 69ms/step
